In [1]:
import syft as sy
from syft.core.node.common.node_service.tff.tff_messages import TFFMessageWithReply
from syft.core.node.common.node_service.ping.ping_messages import PingMessageWithReply
from syft.grid import GridURL
from syft.core.node.common.node_service.simple.simple_messages import NodeRunnableMessageWithReply
from syft.core.node.common.node_service.get_repr.get_repr_messages import GetReprMessage

In [2]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into node... done!


In [3]:
!pip install --quiet --upgrade --user nest-asyncio

In [4]:
import torch as th
x = th.Tensor([1, 2, 3])
y = th.Tensor([1, 2, 3])
x_out = x.send(domain)
# print(x_out)
y_out = y.send(domain)


In [18]:
msg = TFFMessageWithReply("Hello")
# print(y_out)
print(domain.store)
reply_msg = domain.send_immediate_msg_with_reply(msg)
for tensor in domain.store:
    domain.get(tensor)

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'post_init', 'pprint', 'repr_short', 'results', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.object_search.obj_search_service.ObjectSearchReplyMessage'>
[<TensorPointer -> node:d10d704d7e8849488006735d36c342bd>, <TensorPointer -> node:81e7417bb5154b8c89dad48d3fbe6342>, <TensorPointer -> node:95de218269a142ddbea192be25043785>, <TensorPointer -> node:3fd34ebd47934b91be6ac98a549a75f7>, <TensorPointer -> node:d3f4

In [16]:
for key in domain.store:
    print(dir(key))
    print(domain.store[key])

[2022-04-19T14:06:00.520205+0300][CRITICAL][logger]][97572] 'Please pass in a string or int key'


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'post_init', 'pprint', 'repr_short', 'results', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.object_search.obj_search_service.ObjectSearchReplyMessage'>
['T', '__abs__', '__add__', '__and__', '__annotations__', '__class__', '__complex__', '__del__', '__delattr__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt

KeyError: 'Please pass in a string or int key'